In [1]:
import xnat
import os
import shutil

In [2]:
from data_io import filter_series

In [3]:
session = xnat.connect('https://bifrost.icr.ac.uk:8443/XNAT_anonymised/',user='tbarfoot',password='-----')

In [4]:
project = session.projects["MALIMAR_ALL"]
mrSession = project.experiments['20171204_125025_Avanto']
malimarSeries = filter_series(mrSession)

Scan <MrScanData fl3d_vibe_dixon_TRA_15deg 256_pocS_in (50)> does not contain a DICOM resource! oh


In [7]:
malimarSeries.xnat

{'inPhase': [<MrScanData T1 (59)>],
 'outPhase': [<MrScanData T1 OPP (60)>],
 'fat': [<MrScanData T1 FAT (62)>],
 'water': [<MrScanData T1 WAT (63)>],
 'b50': [<MrScanData 50 (23)>, <MrScanData B50 (42)>],
 'b600': [<MrScanData 600 (24)>, <MrScanData B600 (37)>],
 'b900': [<MrScanData 900 (25)>,
  <MrScanData 900.2 (26)>,
  <MrScanData B900 (32)>],
 'adc': [<MrScanData ADC (45)>],
 'diff': []}

In [9]:
for key in malimarSeries.xnat:
    print(key)

inPhase
outPhase
fat
water
b50
b600
b900
adc
diff


In [36]:
complete = (1,1,1,1,1,0,1,1,0)

In [37]:
a = []

In [38]:
for key, c in zip(malimarSeries.xnat,complete):
    a.append(len(malimarSeries.xnat[key])-c)

Complete if:

In [39]:
min(a) > -1 

True

Duplicates if:

In [40]:
max(a) > 0

True

actually no as having b600 would trigger this conditon as well. Write a seperate function that looks at whether len of each list in dictionary is greater than 1

In [41]:
len(malimarSeries.xnat)

9

In [45]:
keys = ['inPhase', 'outPhase', 'fat', 'water', 'b50', 'b600', 'b900', 'adc', 'diff']

In [48]:
import pandas as pd

In [ ]:
pd.D

In [50]:
dict.fromkeys(keys,[pd.DataFrame])

{'inPhase': [pandas.core.frame.DataFrame],
 'outPhase': [pandas.core.frame.DataFrame],
 'fat': [pandas.core.frame.DataFrame],
 'water': [pandas.core.frame.DataFrame],
 'b50': [pandas.core.frame.DataFrame],
 'b600': [pandas.core.frame.DataFrame],
 'b900': [pandas.core.frame.DataFrame],
 'adc': [pandas.core.frame.DataFrame],
 'diff': [pandas.core.frame.DataFrame]}

In [102]:
xnat_paths = {'dix': {'inPhase': [], 'outPhase': [], 'fat': [], 'water': []},
                           'dif': {'b50': [], 'b600': [], 'b900': [], 'adc': [], 'diff': []}}

In [110]:
xnat_paths['dix']['inPhase']

['in']

In [112]:
for group in xnat_paths:
    for series in xnat_paths[group]:
        for item in xnat_paths[group][series]:
            print(item)

in
out
fat
wat
5
6
9
a


In [115]:
xnat_paths = {'dix': {'inPhase': ['in'], 'outPhase': ['out'], 'fat': ['fat'], 'water': ['wat']},
              'dif': {'b50': ['a','b'], 'b600': ['c','d'], 'b900': ['e','f','g'], 'adc': ['h'], 'diff': []}}


In [129]:
complete = ((1, 1, 1, 1),(1, 0, 1, 1, 0))

In [133]:
for comp in complete:
    for c in comp:
        print(c)

1
1
1
1
1
0
1
1
0


In [138]:
for group in xnat_paths:
    for key in xnat_paths[group]:
        print(len(xnat_paths[group][key]))

1
1
1
1
2
2
3
1
0


In [140]:
for group, comp in zip(xnat_paths, complete):
    print(group,comp)

dix (1, 1, 1, 1)
dif (1, 0, 1, 1, 0)


In [141]:
a = []
for group, comp in zip(xnat_paths, complete):
    for key, c in zip(xnat_paths[group], comp):
        a.append(len(xnat_paths[group][key]) - c)
complete = min(a) > -1
duplicates = max(a) > 0

In [142]:
complete

True

In [143]:
duplicates

True

In [144]:
{'dixon': {'inPhase': ['temp/inPhase(1)'], 'outPhase': ['temp/outPhase(1)'], 'fat': ['temp/fat(1)'], 'water': ['temp/water(1)']}, 'diffusion': {'b50': ['temp/b50(1)'], 'b600': [], 'b900': ['temp/b900(1)', 'temp/b900(2)'], 'adc': ['temp/adc(1)'], 'bvals': []}}

{'dixon': {'inPhase': ['temp/inPhase(1)'],
  'outPhase': ['temp/outPhase(1)'],
  'fat': ['temp/fat(1)'],
  'water': ['temp/water(1)']},
 'diffusion': {'b50': ['temp/b50(1)'],
  'b600': [],
  'b900': ['temp/b900(1)', 'temp/b900(2)'],
  'adc': ['temp/adc(1)'],
  'bvals': []}}

In [2]:
paths = {'dixon': {'inPhase': ['temp/inPhase(1)'], 'outPhase': ['temp/outPhase(1)'], 'fat': ['temp/fat(1)'], 'water': ['temp/water(1)']}, 'diffusion': {'b50': ['temp/b50(1)'], 'b600': [], 'b900': ['temp/b900(1)', 'temp/b900(2)'], 'adc': ['temp/adc(1)'], 'bvals': []}}

In [5]:
len(paths)

2

In [146]:
len(paths)

2

In [149]:
import os
import pandas as pd
import pydicom

In [151]:
    fields = ['InstanceNumber', 'SliceLocation', 'Rows', 'Columns', 'SequenceName', 'SeriesNumber']
    local_dcms = {}
    for group in paths:
        for key in paths[group]:
            for path in paths[group][key]:
                rows = []
                for dirName, subdirList, fileList in os.walk(path):
                    for filename in fileList:
                        if ".dcm" in filename.lower():
                            dict1 = {'FilePath': filename}
                            dcm = pydicom.dcmread(os.path.join(dirName, filename), stop_before_pixels=True)
                            for field in fields:
                                dict1.update({field: getattr(dcm, field)})
                            rows.append(dict1)
                local_dcms[0].append(pd.DataFrame(rows, columns=fields))
                
                

KeyError: 0

In [153]:
paths.keys()

dict_keys(['dixon', 'diffusion'])

In [155]:
a = [['i','o','w','f'],['5','6','9','a']]

In [156]:
for group in a:
    for elem in group:
        print(elem)

i
o
w
f
5
6
9
a


In [160]:
b = []

In [158]:
b.append([])

In [159]:
b

[[]]

In [162]:
for i, group in enumerate(a):
    b.append([])
    for elem in group:
        b[i].append(elem)
        

In [2]:
import pandas as pd

In [21]:
df = pd.read_csv('example.csv')

In [22]:
df.head()

,InstanceNumber,SliceLocation,Rows,Columns,SequenceName,SeriesNumber
0,201,-979.714127,416,512,*fl3d2,53
1,202,-974.714127,416,512,*fl3d2,53
2,203,-969.714127,416,512,*fl3d2,53
3,204,-964.714127,416,512,*fl3d2,53
4,205,-959.714127,416,512,*fl3d2,53


In [31]:
not df['InstanceNumber'].is_monotonic

True

In [9]:
df.reset_index(drop=True,inplace=True)

In [11]:
df['InstanceNumber'] = df.index + 1

In [12]:
df

,InstanceNumber,SliceLocation,Rows,Columns,SequenceName,SeriesNumber
0,1,-979.714127,416,512,*fl3d2,53
1,2,-974.714127,416,512,*fl3d2,53
2,3,-969.714127,416,512,*fl3d2,53
3,4,-964.714127,416,512,*fl3d2,53
4,5,-959.714127,416,512,*fl3d2,53
5,6,-954.714127,416,512,*fl3d2,53
6,7,-949.714127,416,512,*fl3d2,53
7,8,-944.714127,416,512,*fl3d2,53
8,9,-939.714127,416,512,*fl3d2,53
9,10,-934.714127,416,512,*fl3d2,53


In [18]:
a = 5

In [19]:
a += 1

In [20]:
a

6

In [33]:
dict1 = {'FilePath': 'boop'}

In [34]:
dict1

{'FilePath': 'boop'}

In [35]:
dict1.update({'InstanceNumber': 6})

In [36]:
dict1

{'FilePath': 'boop', 'InstanceNumber': 6}

In [38]:
dict1.update({'FileWhack':'ehe/egaeh/ehe/'})

In [39]:
dict1

{'FilePath': 'boop', 'InstanceNumber': 6, 'FileWhack': 'ehe/egaeh/ehe/'}

In [41]:
pd.DataFrame([dict1])

,FilePath,FileWhack,InstanceNumber
0,boop,ehe/egaeh/ehe/,6


In [42]:
import pydicom

In [43]:
path = 'temp/inPhase(1)/20181215_152525_Avanto/scans/51-T1_IP/resources/DICOM/files/1.2.840.113564.9.1.2843759204.47.2.5001567892-51-1-x7zcbl.dcm'

In [50]:
ds = pydicom.dcmread(path)

In [51]:
ds.InstanceNumber = 15

In [52]:
ds.save_as('temp/reee.dcm')

In [12]:
import pandas as pd
import numpy as np

In [28]:
df = pd.read_csv('example.csv')

In [29]:
df.columns

Index(['Columns', 'InstanceNumber', 'Rows', 'SequenceName', 'SeriesNumber',
       'SliceLocation', 'path'],
      dtype='object')

In [30]:
import pydicom

In [34]:
def rewrite_instance_number(instance_number,path):
    ds = pydicom.dcmread(path)
    ds.InstanceNumber = instance_number
    ds.save_as('testy.dcm')   
    

In [35]:
df[['InstanceNumber','path']]

,InstanceNumber,path
0,1,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
1,2,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
2,3,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
3,4,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
4,5,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
5,6,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
6,7,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
7,8,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
8,9,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
9,10,temp/inPhase(1)/20181215_152525_Avanto/scans/5...


In [36]:
[myfunc(x, y) for x, y in zip(df['InstanceNumber'], df['path'])]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [87]:
df[['InstanceNumber','path']].apply(myfunc())

TypeError: myfunc() missing 2 required positional arguments: 'path' and 'instance_number'

https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas/41022840#41022840

https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas/55557758#55557758

https://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe

In [39]:
df

,Columns,InstanceNumber,Rows,SequenceName,SeriesNumber,SliceLocation,path
0,512,1,416,*fl3d2,51,-979.714127,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
1,512,2,416,*fl3d2,51,-974.714127,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
2,512,3,416,*fl3d2,51,-969.714127,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
3,512,4,416,*fl3d2,51,-964.714127,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
4,512,5,416,*fl3d2,51,-959.714127,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
5,512,6,416,*fl3d2,51,-954.714127,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
6,512,7,416,*fl3d2,51,-949.714127,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
7,512,8,416,*fl3d2,51,-944.714127,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
8,512,9,416,*fl3d2,51,-939.714127,temp/inPhase(1)/20181215_152525_Avanto/scans/5...
9,512,10,416,*fl3d2,51,-934.714127,temp/inPhase(1)/20181215_152525_Avanto/scans/5...


In [43]:
df['Columns'].nunique() 

1

In [44]:
listy

[['a', 'b', 'c'], ['d', 'e', 'f']]

In [45]:
len(listy)

2

In [46]:
rows = [None] * len(listy)

In [47]:
rows

[None, None]

In [51]:
k = [17, 17, 17, 18]

In [52]:
k.count(k[0]) == len(k)

False

In [1]:
import pandas as pd

In [3]:
pd.DataFrame()

""


In [4]:
sequence = 'dixon'

In [5]:
series = 'inPhase'

In [10]:
dict1 = {'Sequence': sequence}

In [11]:
dict1

{'Sequence': 'dixon'}

In [12]:
dict1.update({'Series': series})

In [13]:
dict1

{'Sequence': 'dixon', 'Series': 'inPhase'}

In [15]:
pd.DataFrame(dict1)

ValueError: If using all scalar values, you must pass an index

In [269]:
selfdf = pd.DataFrame()

In [270]:
paths = {'dixon': {'inPhase': ['temp/inPhase(1)'], 'outPhase': ['temp/outPhase(1)'], 'fat': ['temp/fat(1)'], 'water': ['temp/water(1)']}, 'diffusion': {'b50': ['temp/b50(1)'], 'b600': ['temp/b600(1)'], 'b900': ['temp/b900(1)'], 'adc': ['temp/adc(1)'], 'bvals': []}}

In [271]:
paths

{'dixon': {'inPhase': ['temp/inPhase(1)'],
  'outPhase': ['temp/outPhase(1)'],
  'fat': ['temp/fat(1)'],
  'water': ['temp/water(1)']},
 'diffusion': {'b50': ['temp/b50(1)'],
  'b600': ['temp/b600(1)'],
  'b900': ['temp/b900(1)'],
  'adc': ['temp/adc(1)'],
  'bvals': []}}

In [272]:
import os
import pandas as pd
import pydicom

In [273]:
fields = ['InstanceNumber', 'SliceLocation', 'SliceThickness', 'Rows', 'Columns', 'PixelSpacing']
for sequence in paths:
    for series in paths[sequence]:
        for i, path in enumerate(paths[sequence][series]):
            rows = []
            for dirName, subdirList, fileList in os.walk(path):
                for filename in fileList:
                    if ".dcm" in filename.lower():
                        dict1 = {'Sequence': sequence, 'Series': series+'_'+str(i+1)}
                        dcm_path = os.path.join(dirName, filename)
                        dcm = pydicom.dcmread(dcm_path, stop_before_pixels=True)
                        for field in fields:
                            dict1.update({field: getattr(dcm, field)})
                        dict1.update({'Path': dcm_path})
                        rows.append(dict1)
            df = pd.DataFrame(rows,columns=dict1.keys())
            df.sort_values(by=['SliceLocation'], inplace=True)
            #df.set_index(['Sequence','Series','InstanceNumber'], inplace=True)
            selfdf = selfdf.append(df)

print('List of Dataframes of DICOM headers constructed')

List of Dataframes of DICOM headers constructed


In [274]:
selfdf

,Sequence,Series,InstanceNumber,SliceLocation,SliceThickness,Rows,Columns,PixelSpacing,Path
176,dixon,inPhase_1,1,-979.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/inPhase(1)/20181215_152525_Avanto/scans/5...
84,dixon,inPhase_1,2,-974.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/inPhase(1)/20181215_152525_Avanto/scans/5...
200,dixon,inPhase_1,3,-969.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/inPhase(1)/20181215_152525_Avanto/scans/5...
155,dixon,inPhase_1,4,-964.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/inPhase(1)/20181215_152525_Avanto/scans/5...
221,dixon,inPhase_1,5,-959.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/inPhase(1)/20181215_152525_Avanto/scans/5...
19,dixon,inPhase_1,6,-954.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/inPhase(1)/20181215_152525_Avanto/scans/5...
182,dixon,inPhase_1,7,-949.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/inPhase(1)/20181215_152525_Avanto/scans/5...
197,dixon,inPhase_1,8,-944.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/inPhase(1)/20181215_152525_Avanto/scans/5...
95,dixon,inPhase_1,9,-939.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/inPhase(1)/20181215_152525_Avanto/scans/5...
158,dixon,inPhase_1,10,-934.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/inPhase(1)/20181215_152525_Avanto/scans/5...


In [275]:
selfdf.set_index(['Sequence','Series','InstanceNumber'],inplace=True)

In [276]:
selfdf.loc[('dixon','outPhase_1')]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


,SliceLocation,SliceThickness,Rows,Columns,PixelSpacing,Path
InstanceNumber,,,,,,
201,-979.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
202,-974.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
203,-969.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
204,-964.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
205,-959.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
206,-954.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
207,-949.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
208,-944.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
209,-939.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...


In [277]:
selfdf.loc[('dixon','outPhase_1')].index.is_monotonic

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


False

In [278]:
selfdf.loc[('dixon','outPhase_1')].reset_index()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


,InstanceNumber,SliceLocation,SliceThickness,Rows,Columns,PixelSpacing,Path
0,201,-979.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
1,202,-974.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
2,203,-969.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
3,204,-964.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
4,205,-959.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
5,206,-954.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
6,207,-949.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
7,208,-944.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
8,209,-939.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
9,210,-934.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...


In [258]:
df3 = selfdf.loc[('dixon','outPhase_1')].reset_index()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


In [260]:
df3['InstanceNumber'] = df3.index + 1

In [266]:
df3.set_index('InstanceNumber')

,SliceLocation,SliceThickness,Rows,Columns,PixelSpacing,Path
InstanceNumber,,,,,,
1,-979.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
2,-974.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
3,-969.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
4,-964.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
5,-959.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
6,-954.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
7,-949.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
8,-944.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...
9,-939.714127,5.0,416,512,"[0.83984375, 0.83984375]",temp/outPhase(1)/20181215_152525_Avanto/scans/...


In [267]:
selfdf.loc[('dixon','outPhase_1')] = df3.set_index('InstanceNumber')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2854: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)


In [268]:
selfdf.loc[('dixon','outPhase_1')]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


,SliceLocation,SliceThickness,Rows,Columns,PixelSpacing,Path
InstanceNumber,,,,,,
201,NaN,NaN,NaN,NaN,NaN,NaN
202,NaN,NaN,NaN,NaN,NaN,NaN
203,NaN,NaN,NaN,NaN,NaN,NaN
204,NaN,NaN,NaN,NaN,NaN,NaN
205,NaN,NaN,NaN,NaN,NaN,NaN
206,NaN,NaN,NaN,NaN,NaN,NaN
207,NaN,NaN,NaN,NaN,NaN,NaN
208,NaN,NaN,NaN,NaN,NaN,NaN
209,NaN,NaN,NaN,NaN,NaN,NaN


In [235]:
selfdf.loc[('dixon','inPhase_1')]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


InstanceNumber  SliceLocation  SliceThickness  Rows  \
Sequence Series                                                           
dixon    inPhase_1               1    -979.714127             5.0   416   
         inPhase_1               2    -974.714127             5.0   416   
         inPhase_1               3    -969.714127             5.0   416   
         inPhase_1               4    -964.714127             5.0   416   
         inPhase_1               5    -959.714127             5.0   416   
         inPhase_1               6    -954.714127             5.0   416   
         inPhase_1               7    -949.714127             5.0   416   
         inPhase_1               8    -944.714127             5.0   416   
         inPhase_1               9    -939.714127             5.0   416   
         inPhase_1              10    -934.714127             5.0   416   
         inPhase_1              11    -929.714127             5.0   416   
         inPhase_1              12    -924.714127             5.0   416   
         inPhase_1              13    -919.714127             5.0   416   
         inPhase_1              14    -914.714127             5.0   416   
         inPhase_1              15    -909.714127             5.0   416   
         inPhase_1              16    -904.714127             5.0   416   
         inPhase_1              17    -899.714127             5.0   416   
         inPhase_1              18    -894.714127             5.0   416   
         inPhase_1              19    -889.714127             5.0   416   
         inPhase_1              20    -884.714127             5.0   416   
         inPhase_1              21    -879.714127             5.0   416   
         inPhase_1              22    -874.714127             5.0   416   
         inPhase_1              23    -869.714127             5.0   416   
         inPhase_1              24    -864.714127             5.0   416   
         inPhase_1              25    -859.714127             5.0   416   
         inPhase_1              26    -854.714127             5.0   416   
         inPhase_1              27    -849.714127             5.0   416   
         inPhase_1              28    -844.714127             5.0   416   
         inPhase_1              29    -839.714127             5.0   416   
         inPhase_1              30    -834.714127             5.0   416   
...                            ...            ...             ...   ...   
         inPhase_1             211      70.285873             5.0   416   
         inPhase_1             212      75.285873             5.0   416   
         inPhase_1             213      80.285873             5.0   416   
         inPhase_1             214      85.285873             5.0   416   
         inPhase_1             215      90.285873             5.0   416   
         inPhase_1             216      95.285873             5.0   416   
         inPhase_1             217     100.285873             5.0   416   
         inPhase_1             218     105.285873             5.0   416   
         inPhase_1             219     110.285873             5.0   416   
         inPhase_1             220     115.285873             5.0   416   
         inPhase_1             221     120.285873             5.0   416   
         inPhase_1             222     125.285873             5.0   416   
         inPhase_1             223     130.285873             5.0   416   
         inPhase_1             224     135.285873             5.0   416   
         inPhase_1             225     140.285873             5.0   416   
         inPhase_1             226     145.285873             5.0   416   
         inPhase_1             227     150.285873             5.0   416   
         inPhase_1             228     155.285873             5.0   416   
         inPhase_1             229     160.285873             5.0   416   
         inPhase_1             230     165.285873             5.0   416   
         inPhase_1             231     170.28

In [241]:
selfdf.loc[(slice('dixon','inPhase_1'))]

UnsortedIndexError: 'Key length (1) was greater than MultiIndex lexsort depth (0)'

In [230]:
selfdf.loc[('dixon','inPhase_1'), 'InstanceNumber']

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1494: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


Sequence  Series   
dixon     inPhase_1      1
          inPhase_1      2
          inPhase_1      3
          inPhase_1      4
          inPhase_1      5
          inPhase_1      6
          inPhase_1      7
          inPhase_1      8
          inPhase_1      9
          inPhase_1     10
          inPhase_1     11
          inPhase_1     12
          inPhase_1     13
          inPhase_1     14
          inPhase_1     15
          inPhase_1     16
          inPhase_1     17
          inPhase_1     18
          inPhase_1     19
          inPhase_1     20
          inPhase_1     21
          inPhase_1     22
          inPhase_1     23
          inPhase_1     24
          inPhase_1     25
          inPhase_1     26
          inPhase_1     27
          inPhase_1     28
          inPhase_1     29
          inPhase_1     30
                      ... 
          inPhase_1    211
          inPhase_1    212
          inPhase_1    213
          inPhase_1    214
          inPhase_1    215
        

In [226]:
selfdf.index.levels

FrozenList([['diffusion', 'dixon'], ['adc_1', 'b50_1', 'b600_1', 'b900_1', 'fat_1', 'inPhase_1', 'outPhase_1', 'water_1']])

In [178]:
for thing in selfdf.groupby('Series'):
    print(thing)

('adc_1',       Sequence Series  InstanceNumber  SliceLocation  SliceThickness  Rows  \
82   diffusion  adc_1               1    -979.714127             5.0   208   
77   diffusion  adc_1               2    -974.714127             5.0   208   
212  diffusion  adc_1               3    -969.714127             5.0   208   
134  diffusion  adc_1               4    -964.714127             5.0   208   
15   diffusion  adc_1               5    -959.714127             5.0   208   
48   diffusion  adc_1               6    -954.714127             5.0   208   
104  diffusion  adc_1               7    -949.714127             5.0   208   
24   diffusion  adc_1               8    -944.714127             5.0   208   
140  diffusion  adc_1               9    -939.714127             5.0   208   
81   diffusion  adc_1              10    -934.714127             5.0   208   
206  diffusion  adc_1              11    -929.714127             5.0   208   
57   diffusion  adc_1              12    -924.714127  

In [154]:
selfdf.to_excel('test2.xlsx')

In [123]:
selfdf['InstanceNumber'].ismonotonic

KeyError: 'InstanceNumber'